In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

#Need py38 environment for this to run

import sys
import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

hard = holo.evolution.Sesana_Scattering()


In [ ]:
fig, axes = plt.subplots(2,2,figsize=(15,15), sharex=True, sharey=True)
fs = 30
fname = "eccentricity_evolution_eb"

mrat = [1.0]#, 1./3., 1./9., 1./27.]
ecc = [0.3, 0.6]#[0, 0.3, 0.6, 0.9]
colors = ['lightseagreen', 'gold', 'steelblue', 'firebrick']
sepa_rhard = np.linspace(0.01,2.0,1000)

_K_A = {'qb_1.00' : {'eb_0.30': 0.075,
                     'eb_0.60': 0.121,
                     'eb_0.90': 0.082}
       }

_K_a0 = {'qb_1.00' : {'eb_0.30': 0.151,
                     'eb_0.60': 0.090,
                     'eb_0.90': 0.085}
       }

_K_g = {'qb_1.00' : {'eb_0.30': -1.548,
                     'eb_0.60': -0.895,
                     'eb_0.90': -0.663}
       }

_K_B = {'qb_1.00' : {'eb_0.30': -0.008,
                     'eb_0.60': -0.008,
                     'eb_0.90': -0.004}
       }

for ax, mrat in zip(axes.ravel(), mrat):
    for eb,color in zip(ecc,colors):
        A = _K_A["qb_%.2f" %mrat]["eb_%.2f" %eb]
        a0 = _K_a0["qb_%.2f" %mrat]["eb_%.2f" %eb]
        g = _K_g["qb_%.2f" %mrat]["eb_%.2f" %eb]
        B = _K_B["qb_%.2f" %mrat]["eb_%.2f" %eb]
        use_a = sepa_rhard/a0
        K_manual = A * np.power((1 + use_a), g) + B
        ax.semilogx(sepa_rhard, hard._shm06.K(mrat, sepa_rhard, eb), color=color, label=r'$e_{\rm b} = $' + '%.2f' %eb)
        ax.semilogx(sepa_rhard, K_manual, '--', color=color, label=r'$e_{\rm b} = $' + '%.2f (direct calc)' %eb)
    if ax == axes.ravel()[0]:
        ax.legend(fontsize=0.7*fs, loc='upper left')
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
    ticksize = 15
    tickwidth = 2
    ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    ax.tick_params(axis='both', which='minor', direction='inout', size=0.5*ticksize, width=0.25*tickwidth)
    ax.text(0.4,0.15,'q=%.2f' %mrat, fontsize=fs)
    plt.tight_layout()

fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
fig.supxlabel(r'$a/a_{\rm h}$', fontsize=fs)
fig.supylabel(r'$K(\sigma)$',fontsize=fs)
#plt.xlabel(r'$a/a_{\rm h}$', fontsize=fs)
#plt.ylabel(r'$K(\sigma)$',fontsize=fs)
plt.tight_layout()
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.04), ncol=int(len(ecc)), fontsize=0.8*fs)
plt.tight_layout()
plt.savefig("Sesana06_Fig4.png",facecolor='w')


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,7), sharex=True, sharey=True)
fs = 30
lw = 6
fname = "eccentricity_evolution_eb"

mrats = [1.0]#, 1./3., 1./9., 1./27.]
ecc = [0.3, 0.6, 0.9]#[0, 0.3, 0.6, 0.9]
colors = ['lightseagreen', 'firebrick', 'steelblue', 'gold']
sepa_rhard = np.linspace(0.01,2.0,1000)

_K_A = {'qb_1.00' : {'eb_0.30': 0.075,
                     'eb_0.60': 0.121,
                     'eb_0.90': 0.082}
       }

_K_a0 = {'qb_1.00' : {'eb_0.30': 0.151,
                     'eb_0.60': 0.090,
                     'eb_0.90': 0.085}
       }

_K_g = {'qb_1.00' : {'eb_0.30': -1.548,
                     'eb_0.60': -0.895,
                     'eb_0.90': -0.663}
       }

_K_B = {'qb_1.00' : {'eb_0.30': -0.008,
                     'eb_0.60': -0.008,
                     'eb_0.90': -0.004}
       }

for mrat in mrats:
    for eb,color in zip(ecc,colors):
        A = _K_A["qb_%.2f" %mrat]["eb_%.2f" %eb]
        a0 = _K_a0["qb_%.2f" %mrat]["eb_%.2f" %eb]
        g = _K_g["qb_%.2f" %mrat]["eb_%.2f" %eb]
        B = _K_B["qb_%.2f" %mrat]["eb_%.2f" %eb]
        use_a = sepa_rhard/a0
        K_manual = A * np.power((1 + use_a), g) + B
        ax.semilogx(sepa_rhard, hard._shm06.K(mrat, sepa_rhard, eb), color=color, label=r'$e_{\rm b} = $' + '%.2f (holodeck)' %eb, linewidth = lw, alpha = 0.6)
        ax.semilogx(sepa_rhard, K_manual, '--', color=color, label=r'$e_{\rm b} = $' + '%.2f (direct calc)' %eb, linewidth = lw)
    
    ax.legend(fontsize=0.6*fs, loc='upper right')
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
    ticksize = 15
    tickwidth = 2
    ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    ax.tick_params(axis='both', which='minor', direction='inout', size=0.5*ticksize, width=0.25*tickwidth)
    #ax.text(0.4,0.15,'q=%.2f' %mrat, fontsize=fs)
    plt.tight_layout()

# hide tick and tick label of the big axis
#plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
ax.set_xlabel(r'$a/a_{\rm h}$', fontsize=fs)
ax.set_ylabel(r'$K(\sigma)$',fontsize=fs)
#plt.xlabel(r'$a/a_{\rm h}$', fontsize=fs)
#plt.ylabel(r'$K(\sigma)$',fontsize=fs)
plt.tight_layout()
handles, labels = ax.get_legend_handles_labels()
plt.tight_layout()
plt.savefig("Sesana06_Fig4_qb%.2f.png" %mrat, facecolor='w')
